### 10 corss-validation for CBIS-DDSM, INbreast, MIAS

In [1]:
# Import necessary libraries
import os
import time
import copy
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
device = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:2")

In [ ]:
import mlflow
from datetime import datetime
from torchvision.models import densenet169
import numpy as np

mlflow.set_tracking_uri("http://localhost:5000/")

EXPERIMENT_NAME = "BD_with_SE-attention_and_k-fold_CBIS-DDSM_data_0608"

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)  # check if the experiment is already exist
if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment_id.experiment_id

model = densenet169()

mlflow.start_run(
    nested=True,
    experiment_id=experiment_id,
    run_name=f'test_{datetime.now().strftime("%Y-%m-%d")}',
    tags={
        "type": "breast_density",
        "task": "mammogram_BD_level"
    }
)

In [4]:
# # Initialize the model
# model = models.densenet121(pretrained=True)
# num_ftrs = model.classifier.in_features
# model.classifier = nn.Linear(num_ftrs, 4)
# model = model.to(device)

In [2]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SE_DenseBlock(nn.Module):
    def __init__(self, dense_block, num_channels, reduction=16):
        super().__init__()
        self.dense_block = dense_block
        self.se_block = SELayer(num_channels, reduction)
        
    def forward(self, x):
        return self.se_block(self.dense_block(x))

class SEDenseNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # Load the pretrained densenet
        densenet = models.densenet121(pretrained=True)
        
        # Replace the DenseBlocks with SE_DenseBlocks
        block_channels = [256, 512, 1024, 1024]
        for i, num_channels in enumerate(block_channels, 1):
            dense_block = getattr(densenet.features, f'denseblock{i}')
            se_dense_block = SE_DenseBlock(dense_block, num_channels)
            setattr(densenet.features, f'denseblock{i}', se_dense_block)
        
        # Customize the densenet
        self.features = densenet.features
        self.classifier = nn.Sequential(nn.Linear(1024, num_classes), nn.Softmax(dim=1))

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out




In [3]:
data_dir = '/home/kevinluo/breast_density_classification/BD_data_newdis'

# Define transformations for the training, validation, and testing sets
# Define transformations for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize((640,640)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_test_transforms = transforms.Compose([transforms.Resize((640,640)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])])


# Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms),
    'valid': datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=valid_test_transforms),
    'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=valid_test_transforms)
}

# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=8, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=8, shuffle=True),
    'test': DataLoader(image_datasets['test'], batch_size=8, shuffle=True)
}
# Combine training and validation datasets
full_dataset = torch.utils.data.ConcatDataset([image_datasets["train"], image_datasets["valid"]])

# Extract targets from the dataset
targets = [target for _, target in full_dataset]

# Initialize the KFold object
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


In [5]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SEDenseNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [6]:
num_epochs = 25
n_splits = 10
batch_size = 8
valid_acc_history = []
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for fold, (train_index, valid_index) in enumerate(kf.split(np.zeros(len(full_dataset)), targets)):
    print(f'Fold {fold + 1}/{n_splits}')
    
    train_subset = torch.utils.data.Subset(full_dataset, train_index)
    valid_subset = torch.utils.data.Subset(full_dataset, valid_index)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {'train': train_loader, 'valid': valid_loader}

    model.load_state_dict(best_model_wts)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'valid':
                valid_acc_history.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

print(f'Best valid Acc: {best_acc:.4f}')

# load best model weights
model.load_state_dict(best_model_wts)

# # calculate the mean and standard deviation of the validation accuracies
# mean_acc = np.mean(valid_acc_history)
# std_acc = np.std(valid_acc_history)
# print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# calculate the mean and standard deviation of the validation accuracies
valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
mean_acc = np.mean(valid_acc_history_cpu)
std_acc = np.std(valid_acc_history_cpu)
print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

Fold 1/10
Epoch 1/25
----------
train Loss: 1.2047 Acc: 0.5221
valid Loss: 1.1901 Acc: 0.5249
Epoch 2/25
----------
train Loss: 1.1188 Acc: 0.6089
valid Loss: 1.3492 Acc: 0.3757
Epoch 3/25
----------
train Loss: 1.1047 Acc: 0.6242
valid Loss: 1.0430 Acc: 0.6851
Epoch 4/25
----------
train Loss: 1.0445 Acc: 0.6894
valid Loss: 1.0633 Acc: 0.6796
Epoch 5/25
----------
train Loss: 1.0502 Acc: 0.6802
valid Loss: 1.0773 Acc: 0.6409
Epoch 6/25
----------
train Loss: 1.0347 Acc: 0.7036
valid Loss: 1.2150 Acc: 0.5193
Epoch 7/25
----------
train Loss: 1.0279 Acc: 0.7011
valid Loss: 0.9787 Acc: 0.7514
Epoch 8/25
----------
train Loss: 0.9999 Acc: 0.7392
valid Loss: 1.0803 Acc: 0.6519
Epoch 9/25
----------
train Loss: 1.0452 Acc: 0.6888
valid Loss: 1.0269 Acc: 0.7127
Epoch 10/25
----------
train Loss: 1.0003 Acc: 0.7411
valid Loss: 1.1006 Acc: 0.6243
Epoch 11/25
----------
train Loss: 1.0079 Acc: 0.7306
valid Loss: 0.9787 Acc: 0.7680
Epoch 12/25
----------
train Loss: 1.0152 Acc: 0.7226
valid Loss

TypeError: can't convert cuda:1 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [8]:
# # calculate the mean and standard deviation of the validation accuracies
# valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
# mean_acc = np.mean(valid_acc_history_cpu)
# std_acc = np.std(valid_acc_history_cpu)
# print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')


Mean validation accuracy: 0.8458, Standard deviation: 0.0785


In [14]:
# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))

Test Acc: 0.9056
[[60  0  0  0]
 [ 1 55  4  0]
 [ 0  0 26  6]
 [ 0  0  6 22]]
              precision    recall  f1-score   support

      level1       0.98      1.00      0.99        60
      level2       1.00      0.92      0.96        60
      level3       0.72      0.81      0.76        32
      level4       0.79      0.79      0.79        28

    accuracy                           0.91       180
   macro avg       0.87      0.88      0.87       180
weighted avg       0.91      0.91      0.91       180



In [ ]:
epochs = 50
batch_size = 8
lr = 0.001
mlflow.log_params(
    {
        "epochs": epochs,
        "batch_size": batch_size,
        "lr": lr
    }
)

### train + valid and define 10-fold train model definition

In [ ]:
def train_model_cv(model, criterion, optimizer, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_loss_history = []
    valid_loss_history = []
    valid_acc_history = []

    for fold, (train_index, valid_index) in enumerate(kf.split(full_dataset, targets)):
        print(f'Fold {fold + 1}')
        print('-' * 10)

        # Split the dataset into the training set and the validation set
        train_dataset = torch.utils.data.Subset(full_dataset, train_index)
        valid_dataset = torch.utils.data.Subset(full_dataset, valid_index)

        # Create data loaders for the training and validation sets
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
        dataloaders = {'train': train_loader, 'valid': valid_loader}

        # Reset model parameters
        model.load_state_dict(best_model_wts)

        for epoch in range(num_epochs):
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)
                        # Each epoch has a training and validation phase
            for phase in ['train', 'valid']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                # mlflow.log_metric("{}_loss".format(phase), epoch_loss, step=epoch)
                # mlflow.log_metric("{}_accuracy".format(phase), epoch_acc.item(), step=epoch)

                # deep copy the model
                if phase == 'valid' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

            print()

        print('Best val Acc: {:4f}'.format(best_acc))
        print('-' * 10)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [ ]:
# Set up the loss fxn
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#mlflow紀錄版本
import mlflow

def train_model_cv(model, criterion, optimizer, num_epochs=50):
    # Start MLflow run
    with mlflow.start_run():
        since = time.time()

        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0

        train_loss_history = []
        valid_loss_history = []
        valid_acc_history = []

        for fold, (train_index, valid_index) in enumerate(kf.split(full_dataset, targets)):
            print(f'Fold {fold + 1}')
            print('-' * 10)

            # Log fold index
            mlflow.log_param("fold", fold+1)

            # Split the dataset into the training set and the validation set
            train_dataset = torch.utils.data.Subset(full_dataset, train_index)
            valid_dataset = torch.utils.data.Subset(full_dataset, valid_index)

            # Create data loaders for the training and validation sets
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
            valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
            dataloaders = {'train': train_loader, 'valid': valid_loader}

            # Reset model parameters
            model.load_state_dict(best_model_wts)

            for epoch in range(num_epochs):
                print('Epoch {}/{}'.format(epoch, num_epochs - 1))
                print('-' * 10)

                # Log epoch number
                mlflow.log_param("epoch", epoch+1)

                # Each epoch has a training and validation phase
                for phase in ['train', 'valid']:
                    if phase == 'train':
                        model.train()  # Set model to training mode
                    else:
                        model.eval()   # Set model to evaluate mode

                    running_loss = 0.0
                    running_corrects = 0

                    # Iterate over data.
                    for inputs, labels in tqdm(dataloaders[phase]):
                        inputs = inputs.to(device)
                        labels = labels.to(device)

                        # zero the parameter gradients
                        optimizer.zero_grad()

                        # forward
                        # track history if only in train
                        with torch.set_grad_enabled(phase == 'train'):
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)
                            loss = criterion(outputs, labels)

                            # backward + optimize only if in training phase
                            if phase == 'train':
                                loss.backward()
                                optimizer.step()

                        # statistics
                        running_loss += loss.item() * inputs.size(0)
                        running_corrects += torch.sum(preds == labels.data)

                    epoch_loss = running_loss / len(dataloaders[phase].dataset)
                    epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                    print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                    # Log metrics
                    mlflow.log_metric("{}_loss".format(phase), epoch_loss, step=epoch)
                    mlflow.log_metric("{}_accuracy".format(phase), epoch_acc.item(), step=epoch)

                    # deep copy the model
                    if phase == 'valid' and epoch_acc > best_acc:
                        best_acc = epoch_acc
                        best_model_wts = copy.deepcopy(model.state_dict())

                print()

            print('Best val Acc: {:4f}'.format(best_acc))
            print('-' * 10)
            # Log best validation accuracy
            mlflow.log_metric("best_val_accuracy", best_acc)

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Acc: {:4f}'.format(best_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model



In [ ]:
# Train the model
model = train_model_cv(model, criterion, optimizer, num_epochs=num_epochs)

# Evaluate the model
evaluate_model(model, dataloaders["test"], class_names)

# Save the model
torch.save(model.state_dict(), "0608_10-fold-validation_model_with_se_block.pth")


# INbreast

In [1]:
# Import necessary libraries
import os
import time
import copy
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
device = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:2")

import mlflow
from datetime import datetime
from torchvision.models import densenet169
import numpy as np
## 開不開mlflow, 先不開。
# mlflow.set_tracking_uri("http://localhost:5000/")

# EXPERIMENT_NAME = "BD_with_SE-attention_and_k-fold_CBIS-DDSM_data_0608"

# experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)  # check if the experiment is already exist
# if not experiment_id:
#     experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
# else:
#     experiment_id = experiment_id.experiment_id

# model = densenet169()

# mlflow.start_run(
#     nested=True,
#     experiment_id=experiment_id,
#     run_name=f'test_{datetime.now().strftime("%Y-%m-%d")}',
#     tags={
#         "type": "breast_density",
#         "task": "mammogram_BD_level"
#     }
# )

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SE_DenseBlock(nn.Module):
    def __init__(self, dense_block, num_channels, reduction=16):
        super().__init__()
        self.dense_block = dense_block
        self.se_block = SELayer(num_channels, reduction)
        
    def forward(self, x):
        return self.se_block(self.dense_block(x))

class SEDenseNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # Load the pretrained densenet
        densenet = models.densenet121(pretrained=True)
        
        # Replace the DenseBlocks with SE_DenseBlocks
        block_channels = [256, 512, 1024, 1024]
        for i, num_channels in enumerate(block_channels, 1):
            dense_block = getattr(densenet.features, f'denseblock{i}')
            se_dense_block = SE_DenseBlock(dense_block, num_channels)
            setattr(densenet.features, f'denseblock{i}', se_dense_block)
        
        # Customize the densenet
        self.features = densenet.features
        self.classifier = nn.Sequential(nn.Linear(1024, num_classes), nn.Softmax(dim=1))

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out
#在這裡換待測驗的train, valid, test總資料集
data_dir = '/home/kevinluo/breast_density_classification/INbreast'

# Define transformations for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
        
valid_test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
        
        
# Load the datasets with ImageFolder
image_datasets = {
            'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms),
            'valid': datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=valid_test_transforms),
            'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=valid_test_transforms)
        }
        
# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
            'train': DataLoader(image_datasets['train'], batch_size=8, shuffle=True),
            'valid': DataLoader(image_datasets['valid'], batch_size=8, shuffle=True),
            'test': DataLoader(image_datasets['test'], batch_size=8, shuffle=True)
        }
        # Combine training and validation datasets
full_dataset = torch.utils.data.ConcatDataset([image_datasets["train"], image_datasets["valid"]])
        
# Extract targets from the dataset
targets = [target for _, target in full_dataset]
        
# Initialize the KFold object
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SEDenseNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 80
n_splits = 10
batch_size = 32
valid_acc_history = []
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for fold, (train_index, valid_index) in enumerate(kf.split(np.zeros(len(full_dataset)), targets)):
    print(f'Fold {fold + 1}/{n_splits}')
    
    train_subset = torch.utils.data.Subset(full_dataset, train_index)
    valid_subset = torch.utils.data.Subset(full_dataset, valid_index)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {'train': train_loader, 'valid': valid_loader}

    model.load_state_dict(best_model_wts)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'valid':
                valid_acc_history.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

print(f'Best valid Acc: {best_acc:.4f}')

# load best model weights
model.load_state_dict(best_model_wts)

# # calculate the mean and standard deviation of the validation accuracies
# mean_acc = np.mean(valid_acc_history)
# std_acc = np.std(valid_acc_history)
# print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# calculate the mean and standard deviation of the validation accuracies
#solve the issue that can not print : valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
mean_acc = np.mean(valid_acc_history_cpu)
std_acc = np.std(valid_acc_history_cpu)
print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())  #非常重要 ".cpu().numpy()"
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))


/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1/10
Epoch 1/80
----------
train Loss: 1.1506 Acc: 0.5820
valid Loss: 1.2795 Acc: 0.4381
Epoch 2/80
----------
train Loss: 1.0927 Acc: 0.6413
valid Loss: 1.2207 Acc: 0.5048
Epoch 3/80
----------
train Loss: 1.0541 Acc: 0.6921
valid Loss: 1.4881 Acc: 0.2476
Epoch 4/80
----------
train Loss: 1.0568 Acc: 0.6825
valid Loss: 1.2691 Acc: 0.4667
Epoch 5/80
----------
train Loss: 1.0240 Acc: 0.7196
valid Loss: 1.1852 Acc: 0.5524
Epoch 6/80
----------
train Loss: 1.0216 Acc: 0.7164
valid Loss: 1.3183 Acc: 0.4095
Epoch 7/80
----------
train Loss: 1.0309 Acc: 0.7132
valid Loss: 1.0477 Acc: 0.6667
Epoch 8/80
----------
train Loss: 1.0171 Acc: 0.7206
valid Loss: 1.3757 Acc: 0.3524
Epoch 9/80
----------
train Loss: 1.0088 Acc: 0.7354
valid Loss: 1.4558 Acc: 0.2667
Epoch 10/80
----------
train Loss: 1.0398 Acc: 0.6974
valid Loss: 1.4848 Acc: 0.2571
Epoch 11/80
----------
train Loss: 0.9827 Acc: 0.7587
valid Loss: 1.1240 Acc: 0.6000
Epoch 12/80
----------
train Loss: 0.9657 Acc: 0.7683
valid Loss

In [1]:
# Import necessary libraries
import os
import time
import copy
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
device = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:2")

import mlflow
from datetime import datetime
from torchvision.models import densenet169
import numpy as np
## 開不開mlflow, 先不開。
# mlflow.set_tracking_uri("http://localhost:5000/")

# EXPERIMENT_NAME = "BD_with_SE-attention_and_k-fold_CBIS-DDSM_data_0608"

# experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)  # check if the experiment is already exist
# if not experiment_id:
#     experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
# else:
#     experiment_id = experiment_id.experiment_id

# model = densenet169()

# mlflow.start_run(
#     nested=True,
#     experiment_id=experiment_id,
#     run_name=f'test_{datetime.now().strftime("%Y-%m-%d")}',
#     tags={
#         "type": "breast_density",
#         "task": "mammogram_BD_level"
#     }
# )

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SE_DenseBlock(nn.Module):
    def __init__(self, dense_block, num_channels, reduction=16):
        super().__init__()
        self.dense_block = dense_block
        self.se_block = SELayer(num_channels, reduction)
        
    def forward(self, x):
        return self.se_block(self.dense_block(x))

class SEDenseNet(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        # Load the pretrained densenet
        densenet = models.densenet121(pretrained=True)
        
        # Replace the DenseBlocks with SE_DenseBlocks
        block_channels = [256, 512, 1024, 1024]
        for i, num_channels in enumerate(block_channels, 1):
            dense_block = getattr(densenet.features, f'denseblock{i}')
            se_dense_block = SE_DenseBlock(dense_block, num_channels)
            setattr(densenet.features, f'denseblock{i}', se_dense_block)
        
        # Customize the densenet
        self.features = densenet.features
        self.classifier = nn.Sequential(nn.Linear(1024, num_classes), nn.Softmax(dim=1))

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out
#在這裡換待測驗的train, valid, test總資料集
data_dir = '/home/kevinluo/breast_density_classification/MIAS_BD_data'

# Define transformations for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
        
valid_test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
        
        
# Load the datasets with ImageFolder
image_datasets = {
            'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms),
            'valid': datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=valid_test_transforms),
            'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=valid_test_transforms)
        }
        
# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
            'train': DataLoader(image_datasets['train'], batch_size=8, shuffle=True),
            'valid': DataLoader(image_datasets['valid'], batch_size=8, shuffle=True),
            'test': DataLoader(image_datasets['test'], batch_size=8, shuffle=True)
        }
        # Combine training and validation datasets
full_dataset = torch.utils.data.ConcatDataset([image_datasets["train"], image_datasets["valid"]])
        
# Extract targets from the dataset
targets = [target for _, target in full_dataset]
        
# Initialize the KFold object
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SEDenseNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 80
n_splits = 10
batch_size = 32
valid_acc_history = []
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for fold, (train_index, valid_index) in enumerate(kf.split(np.zeros(len(full_dataset)), targets)):
    print(f'Fold {fold + 1}/{n_splits}')
    
    train_subset = torch.utils.data.Subset(full_dataset, train_index)
    valid_subset = torch.utils.data.Subset(full_dataset, valid_index)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {'train': train_loader, 'valid': valid_loader}

    model.load_state_dict(best_model_wts)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'valid':
                valid_acc_history.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

print(f'Best valid Acc: {best_acc:.4f}')

# load best model weights
model.load_state_dict(best_model_wts)

# # calculate the mean and standard deviation of the validation accuracies
# mean_acc = np.mean(valid_acc_history)
# std_acc = np.std(valid_acc_history)
# print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# calculate the mean and standard deviation of the validation accuracies
#solve the issue that can not print : valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
mean_acc = np.mean(valid_acc_history_cpu)
std_acc = np.std(valid_acc_history_cpu)
print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())  #非常重要 ".cpu().numpy()"
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))


/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1/10
Epoch 1/80
----------
train Loss: 0.9225 Acc: 0.6054
valid Loss: 1.0181 Acc: 0.4483
Epoch 2/80
----------
train Loss: 0.8311 Acc: 0.6935
valid Loss: 0.9354 Acc: 0.6207
Epoch 3/80
----------
train Loss: 0.8493 Acc: 0.6782
valid Loss: 1.0821 Acc: 0.3793
Epoch 4/80
----------
train Loss: 0.8127 Acc: 0.7356
valid Loss: 1.0607 Acc: 0.4483
Epoch 5/80
----------
train Loss: 0.7822 Acc: 0.7625
valid Loss: 0.8944 Acc: 0.6897
Epoch 6/80
----------
train Loss: 0.7473 Acc: 0.8084
valid Loss: 0.9749 Acc: 0.5517
Epoch 7/80
----------
train Loss: 0.7900 Acc: 0.7510
valid Loss: 0.8925 Acc: 0.6552
Epoch 8/80
----------
train Loss: 0.7668 Acc: 0.7893
valid Loss: 1.1723 Acc: 0.3793
Epoch 9/80
----------
train Loss: 0.7295 Acc: 0.8199
valid Loss: 0.7697 Acc: 0.7931
Epoch 10/80
----------
train Loss: 0.7791 Acc: 0.7701
valid Loss: 1.0365 Acc: 0.5172
Epoch 11/80
----------
train Loss: 0.7342 Acc: 0.8123
valid Loss: 0.7965 Acc: 0.7586
Epoch 12/80
----------
train Loss: 0.6948 Acc: 0.8582
valid Loss

In [2]:
# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))

Test Acc: 1.0000
[[11  0  0]
 [ 0  9  0]
 [ 0  0  9]]
              precision    recall  f1-score   support

       Dense       1.00      1.00      1.00        11
       Fatty       1.00      1.00      1.00         9
   Glandular       1.00      1.00      1.00         9

    accuracy                           1.00        29
   macro avg       1.00      1.00      1.00        29
weighted avg       1.00      1.00      1.00        29



In [1]:
# Import necessary libraries
import os
import time
import copy
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cuda:2")

import mlflow
from datetime import datetime
from torchvision.models import densenet169
import numpy as np
## 開不開mlflow, 先不開。
# mlflow.set_tracking_uri("http://localhost:5000/")

# EXPERIMENT_NAME = "BD_with_SE-attention_and_k-fold_CBIS-DDSM_data_0608"

# experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)  # check if the experiment is already exist
# if not experiment_id:
#     experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
# else:
#     experiment_id = experiment_id.experiment_id

# model = densenet169()

# mlflow.start_run(
#     nested=True,
#     experiment_id=experiment_id,
#     run_name=f'test_{datetime.now().strftime("%Y-%m-%d")}',
#     tags={
#         "type": "breast_density",
#         "task": "mammogram_BD_level"
#     }
# )

class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SE_DenseBlock(nn.Module):
    def __init__(self, dense_block, num_channels, reduction=16):
        super().__init__()
        self.dense_block = dense_block
        self.se_block = SELayer(num_channels, reduction)
        
    def forward(self, x):
        return self.se_block(self.dense_block(x))

class SEDenseNet(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # Load the pretrained densenet
        densenet = models.densenet121(pretrained=True)
        
        # Replace the DenseBlocks with SE_DenseBlocks
        block_channels = [256, 512, 1024, 1024]
        for i, num_channels in enumerate(block_channels, 1):
            dense_block = getattr(densenet.features, f'denseblock{i}')
            se_dense_block = SE_DenseBlock(dense_block, num_channels)
            setattr(densenet.features, f'denseblock{i}', se_dense_block)
        
        # Customize the densenet
        self.features = densenet.features
        self.classifier = nn.Sequential(nn.Linear(1024, num_classes), nn.Softmax(dim=1))

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.classifier(out)
        return out
#在這裡換待測驗的train, valid, test總資料集
data_dir = '/home/kevinluo/breast_density_classification/BD_data_newdis'

# Define transformations for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
        
valid_test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406],
                                                                [0.229, 0.224, 0.225])])
        
        
# Load the datasets with ImageFolder
image_datasets = {
            'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms),
            'valid': datasets.ImageFolder(os.path.join(data_dir, 'valid'), transform=valid_test_transforms),
            'test': datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=valid_test_transforms)
        }
        
# Using the image datasets and the trainforms, define the dataloaders
dataloaders = {
            'train': DataLoader(image_datasets['train'], batch_size=8, shuffle=True),
            'valid': DataLoader(image_datasets['valid'], batch_size=8, shuffle=True),
            'test': DataLoader(image_datasets['test'], batch_size=8, shuffle=True)
        }
        # Combine training and validation datasets
full_dataset = torch.utils.data.ConcatDataset([image_datasets["train"], image_datasets["valid"]])
        
# Extract targets from the dataset
targets = [target for _, target in full_dataset]
        
# Initialize the KFold object
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SEDenseNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

num_epochs = 80
n_splits = 10
batch_size = 32
valid_acc_history = []
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for fold, (train_index, valid_index) in enumerate(kf.split(np.zeros(len(full_dataset)), targets)):
    print(f'Fold {fold + 1}/{n_splits}')
    
    train_subset = torch.utils.data.Subset(full_dataset, train_index)
    valid_subset = torch.utils.data.Subset(full_dataset, valid_index)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    dataloaders = {'train': train_loader, 'valid': valid_loader}

    model.load_state_dict(best_model_wts)
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'valid':
                valid_acc_history.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())

print(f'Best valid Acc: {best_acc:.4f}')

# load best model weights
model.load_state_dict(best_model_wts)

# # calculate the mean and standard deviation of the validation accuracies
# mean_acc = np.mean(valid_acc_history)
# std_acc = np.std(valid_acc_history)
# print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# calculate the mean and standard deviation of the validation accuracies
#solve the issue that can not print : valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
valid_acc_history_cpu = [acc.cpu().numpy() for acc in valid_acc_history]
mean_acc = np.mean(valid_acc_history_cpu)
std_acc = np.std(valid_acc_history_cpu)
print(f'Mean validation accuracy: {mean_acc:.4f}, Standard deviation: {std_acc:.4f}')

# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())  #非常重要 ".cpu().numpy()"
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))


/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1/10
Epoch 1/80
----------
train Loss: 1.0314 Acc: 0.7066
valid Loss: 1.1800 Acc: 0.5580
Epoch 2/80
----------
train Loss: 0.9668 Acc: 0.7737
valid Loss: 1.1125 Acc: 0.6243
Epoch 3/80
----------
train Loss: 0.9541 Acc: 0.7823
valid Loss: 0.9440 Acc: 0.8011
Epoch 4/80
----------
train Loss: 0.9361 Acc: 0.8057
valid Loss: 1.1298 Acc: 0.6133
Epoch 5/80
----------
train Loss: 0.9440 Acc: 0.7958
valid Loss: 1.0518 Acc: 0.6906
Epoch 6/80
----------
train Loss: 0.9320 Acc: 0.8087
valid Loss: 0.9566 Acc: 0.7956
Epoch 7/80
----------
train Loss: 0.9291 Acc: 0.8087
valid Loss: 1.0788 Acc: 0.6519
Epoch 8/80
----------
train Loss: 0.9146 Acc: 0.8253
valid Loss: 1.0438 Acc: 0.7072
Epoch 9/80
----------
train Loss: 0.8973 Acc: 0.8426
valid Loss: 0.9428 Acc: 0.8011
Epoch 10/80
----------
train Loss: 0.8999 Acc: 0.8456
valid Loss: 1.0105 Acc: 0.7127
Epoch 11/80
----------
train Loss: 0.8987 Acc: 0.8413
valid Loss: 0.9348 Acc: 0.8066
Epoch 12/80
----------
train Loss: 0.8967 Acc: 0.8469
valid Loss

In [2]:
# set model to evaluation mode
model.eval()

test_acc = 0.0
all_preds = []
all_labels = []

# Iterate over test data
for inputs, labels in dataloaders['valid']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
    
    # statistics
    test_acc += torch.sum(preds == labels.data)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.data.cpu().numpy())

test_acc /= len(dataloaders['valid'].dataset)

print(f'Test Acc: {test_acc:.4f}')

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=image_datasets['valid'].classes))

Test Acc: 1.0000
[[60  0  0  0]
 [ 0 60  0  0]
 [ 0  0 32  0]
 [ 0  0  0 28]]
              precision    recall  f1-score   support

      level1       1.00      1.00      1.00        60
      level2       1.00      1.00      1.00        60
      level3       1.00      1.00      1.00        32
      level4       1.00      1.00      1.00        28

    accuracy                           1.00       180
   macro avg       1.00      1.00      1.00       180
weighted avg       1.00      1.00      1.00       180

